In [ ]:
import pandas as pd
import os

# Ruta a la carpeta donde están los archivos CSV
ruta_csv = "Data/csv"  # Cambia esta ruta a tu carpeta de CSVs

# Lista para almacenar los DataFrames
dfs = []

# Iterar sobre todos los archivos CSV en la carpeta
for archivo in os.listdir(ruta_csv):
    if archivo.endswith(".csv"):
        archivo_path = os.path.join(ruta_csv, archivo)
        print(f"Levantando archivo: {archivo_path}")
        
        # Leer el archivo CSV y agregarlo a la lista
        df = pd.read_csv(archivo_path)
        dfs.append(df)

# Concatenar todos los DataFrames en uno solo
df_completo = pd.concat(dfs, ignore_index=True)

# Mostrar la información del DataFrame
print("Información del DataFrame:")
print(df_completo.info())

# Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
print(df_completo.head())


In [3]:
# Guardar el DataFrame en un nuevo archivo CSV
df_completo.to_csv("Data/csv_completo.csv", index=False)

In [15]:
df_completo = pd.read_csv("Data/csv_completo.csv")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_8072\2868210097.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_completo = pd.read_csv("Data/csv_completo.csv")


In [16]:
df_completo.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)',
       'sale_date'],
      dtype='object')

In [17]:
import numpy as np

# Asegurarnos de que el DataFrame está cargado y tiene la columna 'sale_date'
df_completo = df_completo.copy()

# Definir los meses del rango diciembre 2023 - noviembre 2024
inicio = pd.to_datetime("2023-12-01")
fin = pd.to_datetime("2024-11-30")

# Generar los meses y pesos (más peso en los meses de invierno: diciembre, enero, febrero)
meses = [12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
pesos = [2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # Sesgo hacia los meses de invierno

# Calcular el número de registros por mes (aproximadamente)
num_registros = len(df_completo)
num_registros_por_mes = num_registros // len(meses)

# Crear una lista para almacenar las nuevas fechas
nuevas_fechas = []

# Asignar fechas a cada registro con el sesgo
for mes, peso in zip(meses, pesos):
    # Generar fechas aleatorias para el mes específico
    fechas_mes = pd.date_range(start=f"{inicio.year}-{mes:02d}-01", 
                               end=f"{inicio.year}-{mes:02d}-{pd.Timestamp(year=inicio.year, month=mes, day=1).days_in_month}",
                               freq="D")
    
    # Aplicar el peso para generar más fechas en los meses de invierno
    num_fechas = int(num_registros_por_mes * peso / sum(pesos))  # Ajustamos el número de registros
    fechas_aleatorias = np.random.choice(fechas_mes, num_fechas, replace=True)
    
    nuevas_fechas.extend(fechas_aleatorias)

# Si las fechas generadas son menos que el total de registros, duplicamos las fechas aleatorias
if len(nuevas_fechas) < num_registros:
    nuevas_fechas = np.random.choice(nuevas_fechas, num_registros, replace=True)

# Asignar las nuevas fechas al DataFrame
df_completo['sale_date'] = nuevas_fechas[:len(df_completo)]

# Verificar la distribución de fechas en los registros
print(df_completo['sale_date'].value_counts())

# Verificar si la columna 'Sale_Dollars' existe y si no, mostrar un mensaje
if 'Sale (Dollars)' not in df_completo.columns:
    print("La columna 'Sale (Dollars)' no se encuentra en el DataFrame.")
else:
    # Extraer el mes de la columna 'sale_date'
    df_completo['Month'] = df_completo['sale_date'].dt.month
    
    # Verificar las primeras filas del DataFrame
    print(df_completo[['sale_date', 'Month', 'Sale (Dollars)']].head())



sale_date
2023-02-27    2502
2023-02-03    2342
2023-02-12    2252
2023-02-11    2251
2023-02-21    2239
              ... 
2023-08-08     704
2023-05-31     701
2023-03-30     697
2023-10-31     693
2023-10-01     685
Name: count, Length: 365, dtype: int64
   sale_date  Month Sale (Dollars)
0 2023-06-03      6         $28.08
1 2023-05-21      5         $66.24
2 2023-11-09     11         $22.20
3 2023-02-05      2         $67.14
4 2023-02-26      2        $175.92


In [18]:
# Guardar el DataFrame en un nuevo archivo CSV
df_completo.to_csv("Data/csv_completo.csv", index=False)